<a id='top'></a>
<img align="center" src="https://ieltsunlocked.files.wordpress.com/2019/03/choices-1.gif" width="200" />
<p style="text-align: center;">
    <font size='5' type='bold'>
        <b>Рекомендация тарифов</b>
    </font>
</p>

### Содержание

<a href='#section_0'>Задачи проекта</a>

<a href='#section_1'>1. Откройте и изучите файл</a>

<a href='#section_2'>2. Разбейте данные на выборки</a>
    
<a href='#section_3'>3. Исследуйте модели</a>

* <a href='#section_3_1'>Дерево решений</a>
* <a href='#section_3_2'>Случайный лес</a>
* <a href='#section_3_3'>Логистическая регрессия</a>

<a href='#section_4'>4. Проверьте модель на тестовой выборке</a>

<a href='#section_5'>5. (бонус) Проверьте модели на адекватность</a>
    
<a href='#section_end'>Чек-лист готовности проекта</a>

<a id='section_0'></a>
### Задачи проекта
В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

<a id='section_1'></a>
### 1. Откройте и изучите файл

In [2]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [3]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [21]:
df.head(3)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0


Датасет состоит из 3214 объектов и 5 признаков. Целевой признак для нашей задачи – `is_ultra`. Модель должна предсказывать значение **1** если клиенту нужно предложить тариф "Ультра" или **0** если тариф "Смарт". Наш целевой признак является категориальным, а значит мы будем решать задачу классификации. 

Таблица *df* (информация о поведении пользователей) содержит следующие данные:
- **calls** — количество звонков
- **minutes** — суммарная длительность звонков в минутах
- **messages** — количество sms-сообщений
- **mb_used** — израсходованный интернет-трафик в Мб
- **is_ultra** — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0)

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц.

Всего в таблице 5 столбцов, типы данных: 
- **целые числа** у 1 столбца
- **вещественные числа** (с "плавающей" точкой) у 4

Пропуски не представлены, так как предобработка данных уже проведена.

<a href='#top'>Back to top</a>

<a id='section_2'></a>
### 2. Разбейте данные на выборки
Разделим исходные данные на обучающую, валидационную и тестовую выборки. Размеры тестового и валидационного наборов сделаем равными.

In [22]:
# Отведем 60% под обучающую выборку
df_train, df_valid = train_test_split(df, test_size=0.4, random_state=54321)

# Оставшиеся 40% поделим на валидационную и на тестовую
df_valid, df_test = train_test_split(df_valid, test_size=0.5, random_state=54321)

Получили подготовленный датасет с 60% обучающей выборки, 20% валидационной и 20% тестовой.

<a href='#top'>Back to top</a>

<a id='section_3'></a>
### 3. Исследуйте модели
Исследуем качество разных моделей, меняя гиперпараметры. Для задачи классификации рассмотрим следующие:
* Дерево решений
* Случайный лес
* Логистическая регрессия

In [23]:
features_train = df_train.drop(['is_ultra'], axis=1) # фичи обучающей выборки
target_train = df_train['is_ultra'] # целевые признаки обучающей выборки

features_valid = df_valid.drop(['is_ultra'], axis=1) # фичи валидационной выборки
target_valid = df_valid['is_ultra'] # целевые признаки валидационной выборки

<a id='section_3_1'></a>
#### Дерево решений

Исследуем качество модели решающего дерева с различной глубиной

In [24]:
for depth in range(2, 17, 2):
    model = DecisionTreeClassifier(random_state=54321, max_depth=depth)
    model.fit(features_train, target_train) # обучим модель
    predictions = model.predict(features_valid) # получим предсказания модели
    result = accuracy_score(target_valid, predictions) # посчитаем качество модели
    print('Качество модели с глубиной дерева = {}:'.format(depth), result)

Качество модели с глубиной дерева = 2: 0.7418351477449455
Качество модели с глубиной дерева = 4: 0.744945567651633
Качество модели с глубиной дерева = 6: 0.7542768273716952
Качество модели с глубиной дерева = 8: 0.7511664074650077
Качество модели с глубиной дерева = 10: 0.7822706065318819
Качество модели с глубиной дерева = 12: 0.7480559875583204
Качество модели с глубиной дерева = 14: 0.7356143079315708
Качество модели с глубиной дерева = 16: 0.7293934681181959


Наилучшим качеством обладает модель с глубиной дерева = 10

<a href='#top'>Back to top</a>

<a id='section_3_2'></a>
#### Случайный лес

In [26]:
for est in range(10, 101, 10):
    model = RandomForestClassifier(random_state=54321, n_estimators=est, max_depth=10)
    model.fit(features_train, target_train) # обучим модель
    predictions = model.predict(features_valid) # получим предсказания модели
    result = accuracy_score(target_valid, predictions) # посчитаем качество модели
    print('Качество модели с количеством деревьев = {}:'.format(est), result)

Качество модели с количеством деревьев = 10: 0.80248833592535
Качество модели с количеством деревьев = 20: 0.7869362363919129
Качество модели с количеством деревьев = 30: 0.7900466562986003
Качество модели с количеством деревьев = 40: 0.7916018662519441
Качество модели с количеством деревьев = 50: 0.7916018662519441
Качество модели с количеством деревьев = 60: 0.7947122861586314
Качество модели с количеством деревьев = 70: 0.7947122861586314
Качество модели с количеством деревьев = 80: 0.7962674961119751
Качество модели с количеством деревьев = 90: 0.7978227060653188
Качество модели с количеством деревьев = 100: 0.7978227060653188


Наилучшим качеством обладает модель с количеством деревьев = 10. При этом стоит отметить, что качество модели случайного леса (0.80) достаточно близко к значению качества модели решающего дерева (0.78).

<a href='#top'>Back to top</a>

<a id='section_3_3'></a>
#### Логистическая регрессия

In [28]:
solvers = ['newton-cg', 'lbfgs', 'liblinear', 'sag'] # обозначим набор алгоритмов, использующийся для оптимизации (solvers)

for solver in solvers:
    model = LogisticRegression(random_state=54321, solver=solver)
    model.fit(features_train, target_train) # обучим модель
    predictions = model.predict(features_valid) # получим предсказания модели
    result = accuracy_score(target_valid, predictions) # посчитаем качество модели
    print('Качество модели с параметром = {}:'.format(solver), result)

Качество модели с параметром = newton-cg: 0.7076205287713841
Качество модели с параметром = lbfgs: 0.6749611197511665
Качество модели с параметром = liblinear: 0.6780715396578538
Качество модели с параметром = sag: 0.6516329704510109


/opt/conda/lib/python3.7/site-packages/scipy/optimize/linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.7/site-packages/scipy/optimize/linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.7/site-packages/scipy/optimize/linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.7/site-packages/scipy/optimize/linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.7/site-packages/scipy/optimize/linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did

Наилучшим качеством обладает модель с параметром алгоритма для оптимизации `newton-cg` - 0.71. Тем не менее, качество модели логистической регрессии ниже качества моделей решающего дерева и случайного леса.

#### Вывод
Исследовав качество разных моделей со сменой гиперпараметров, выявили:
* Качество модели решающего дерева (с оптимальным параметром по глубине (10)) - 0.78
* Качество модели случайного леса (с оптимальным параметром по количеству деревьев (10)) - 0.80
* Качество модели логистической регрессии (с оптимальным алгоритмом для оптимизации `newton-cg` - 0.71

Модель случайного леса - оптимальна. Проверим ее качество на тестовой выборке.

<a href='#top'>Back to top</a>

<a id='section_4'></a>
### 4. Проверьте модель на тестовой выборке
Проверим качество модели на тестовой выборке.

In [29]:
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

model = RandomForestClassifier(random_state=54321, n_estimators=10, max_depth=10)
model.fit(features_train, target_train)
predictions = model.predict(features_test)
result = accuracy_score(target_test, predictions)
print('Качество модели случайного леса:', result)

Качество модели случайного леса: 0.8242612752721618


На тестовой выборке получили качество модели даже выше, чем на обучающей - 0.82. Значит модель работает.

<a href='#top'>Back to top</a>

<a id='section_5'></a>
### 5. (бонус) Проверьте модели на адекватность

Качество модели случайного леса (с оптимальным параметром по количеству деревьев (10)) - 0.80. На тестовой выборке получили качество модели - 0.82. Как видим значения очень близки к друг другу. Однако выбор методики проверки модели на адекватность - задача не тривиальная.

<a href='#top'>Back to top</a>